In [135]:
import pandas as pd
import plotly.express as px

def plot_flame_horizontal(df, width=1100, height=800):
    df = df.copy()

    # fill NA with ROOT
    df["parentId"] = df["parentId"].fillna("ROOT")

    # redirect missing parents to ROOT
    valid_ids = set(df["ID"])
    df.loc[~df["parentId"].isin(valid_ids) & (df["parentId"] != "ROOT"), "parentId"] = "ROOT"

    # Smart text wrapping (match plot_sunburst style)
    def wrap_text_and_orient(row):
        text = row["Term"]
        parent_id = row["parentId"]

        if text == "Reactome pathways":
            return text, "horizontal"

        if parent_id == "ROOT":
            orientation = "radial"
        else:
            orientation = "tangential"

        if parent_id == "ROOT":
            if len(text) <= 10:
                wrapped_text = text
            elif len(text) <= 10:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = (
                        " ".join(words[:third])
                        + "<br>"
                        + " ".join(words[third : 2 * third])
                        + "<br>"
                        + " ".join(words[2 * third :])
                    )
                elif len(words) == 2:
                    wrapped_text = words[0] + "<br>" + words[1]
                else:
                    wrapped_text = text
        else:
            if len(text) <= 20:
                wrapped_text = text
            elif len(text) <= 30:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = (
                        " ".join(words[:third])
                        + "<br>"
                        + " ".join(words[third : 3 * third])
                        + "<br>"
                        + " ".join(words[3 * third :])
                    )
                elif len(words) == 9:
                    fourth = len(words) // 9
                    wrapped_text = (
                        " ".join(words[:fourth])
                        + "<br>"
                        + " ".join(words[fourth : 9 * fourth])
                        + "<br>"
                        + " ".join(words[9 * fourth :])
                    )
                else:
                    wrapped_text = text

        return wrapped_text, orientation

    wrapped_data = df.apply(wrap_text_and_orient, axis=1, result_type="expand")
    df["wrapped_term"] = wrapped_data[0]
    df["text_orientation"] = wrapped_data[1]

    # add root node
    root = pd.DataFrame(
        [
            {
                "ID": "ROOT",
                "parentId": "",
                "wrapped_term": "Reactome pathways",
                "Term": "Reactome pathways",
                "nes": 0,
                "text_orientation": "horizontal",
            }
        ]
    )
    df = pd.concat([df, root], ignore_index=True)

    # Colorblind-friendly diverging scale
    color_scale = ["#0571b0", "#92c5de", "#f7f7f7", "#f4a582", "#ca0020"]

    # Per-node text colors: dark grey for center, grey for nes==0, white otherwise
    text_colors = [
        "#2F4F4F" if term == "Reactome pathways" else ("#A9A9A9" if (pd.notna(nes_val) and float(nes_val) == 0) else "white")
        for term, nes_val in zip(df["Term"], df["nes"])
    ]

    # Hide center label; we'll render a custom annotation instead
    df.loc[df["ID"].eq("ROOT"), "wrapped_term"] = " "

    # Icicle chart (horizontal flame-like)
    fig = px.icicle(
        df,
        names="wrapped_term",
        ids="ID",
        parents="parentId",
        values=None,
        color="nes",
        color_continuous_scale=color_scale,
        color_continuous_midpoint=0,
        branchvalues="total",
        width=width,
        height=height,
    )

    fig.update_layout(
        margin=dict(t=50, l=50, r=50, b=50),
        coloraxis_colorbar=dict(
            title=dict(text="NES", font=dict(size=16)),
            tickvals=[-5, -2.5, 0, 2.5, 5],
            tickfont=dict(size=14),
            ticks="outside",
            thickness=20,
            len=0.6,
        ),
        font=dict(size=14),
    )

    fig.update_traces(
        textinfo="label",
        texttemplate="<b>%{label}</b>",
        textfont=dict(size=14, family="Arial", color=text_colors),
        hovertemplate="<b>%{customdata[0]}</b><br>NES: %{color:.2f}<extra></extra>",
        customdata=df[["Term"]],
        tiling=dict(orientation="h") if hasattr(type(fig.data[0]), "tiling") else None,
    )

    return fig



In [6]:
import pandas as pd

In [145]:
import math
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


def plot_circle_packing(df, width=1100, height=800):
    df = df.copy()

    # fill NA with ROOT
    df["parentId"] = df["parentId"].fillna("ROOT")

    # redirect missing parents to ROOT
    valid_ids = set(df["ID"])
    df.loc[~df["parentId"].isin(valid_ids) & (df["parentId"] != "ROOT"), "parentId"] = "ROOT"

    # Smart text wrapping (same logic as plot_sunburst)
    def wrap_text_and_orient(row):
        text = row["Term"]
        parent_id = row["parentId"]

        if text == "Reactome pathways":
            return text, "horizontal"

        if parent_id == "ROOT":
            orientation = "radial"
        else:
            orientation = "tangential"

        if parent_id == "ROOT":
            if len(text) <= 10:
                wrapped_text = text
            elif len(text) <= 10:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = (
                        " ".join(words[:third])
                        + "<br>"
                        + " ".join(words[third : 2 * third])
                        + "<br>"
                        + " ".join(words[2 * third :])
                    )
                elif len(words) == 2:
                    wrapped_text = words[0] + "<br>" + words[1]
                else:
                    wrapped_text = text
        else:
            if len(text) <= 20:
                wrapped_text = text
            elif len(text) <= 30:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = (
                        " ".join(words[:third])
                        + "<br>"
                        + " ".join(words[third : 3 * third])
                        + "<br>"
                        + " ".join(words[3 * third :])
                    )
                elif len(words) == 9:
                    fourth = len(words) // 9
                    wrapped_text = (
                        " ".join(words[:fourth])
                        + "<br>"
                        + " ".join(words[fourth : 9 * fourth])
                        + "<br>"
                        + " ".join(words[9 * fourth :])
                    )
                else:
                    wrapped_text = text
        return wrapped_text, orientation

    wrapped_data = df.apply(wrap_text_and_orient, axis=1, result_type="expand")
    df["wrapped_term"] = wrapped_data[0]
    df["text_orientation"] = wrapped_data[1]

    # add root node
    root = pd.DataFrame(
        [
            {
                "ID": "ROOT",
                "parentId": "",
                "wrapped_term": "Reactome pathways",
                "Term": "Reactome pathways",
                "nes": 0,
                "text_orientation": "horizontal",
            }
        ]
    )
    df = pd.concat([df, root], ignore_index=True)

    # Build tree structures
    children_map = {}
    value_map = {}
    term_map = {}
    nes_map = {}
    for _, row in df.iterrows():
        node_id = row["ID"]
        parent_id = row["parentId"]
        children_map.setdefault(parent_id, []).append(node_id)
        children_map.setdefault(node_id, [])
        term_map[node_id] = row["wrapped_term"]
        nes_map[node_id] = row.get("nes", np.nan)
        value_map[node_id] = float(abs(row.get("nes", 0)) if pd.notna(row.get("nes", np.nan)) else 0.0) + 0.01

    # Aggregate values bottom-up
    visited = set()
    def aggregate(node_id):
        if node_id in visited:
            return value_map[node_id]
        visited.add(node_id)
        if children_map.get(node_id):
            child_vals = [aggregate(c) for c in children_map[node_id]]
            value_map[node_id] = max(sum(child_vals), 0.01)
        return value_map[node_id]

    aggregate("ROOT")

    # Radii proportional to sqrt(value)
    raw_radii = {nid: math.sqrt(max(value_map[nid], 1e-6)) for nid in value_map}

    # Place first-level groups (children of ROOT) around a circle
    groups = [nid for nid in children_map.get("ROOT", []) if nid != "ROOT"]
    if not groups:
        groups = [nid for nid in df["ID"] if nid != "ROOT"]

    group_radii = [raw_radii[g] for g in groups]
    total_r = sum(group_radii) + 1e-6
    angles = np.cumsum([0] + [2 * math.pi * (r / total_r) for r in group_radii[:-1]])

    layout_radius = 0.65
    group_scale = (layout_radius * 0.9) / (max(group_radii) if group_radii else 1.0)

    node_pos = {}
    node_rad = {}

    for g, r, ang in zip(groups, group_radii, angles):
        Rg = r * group_scale
        xg = layout_radius * math.cos(ang)
        yg = layout_radius * math.sin(ang)
        node_pos[g] = (xg, yg)
        node_rad[g] = Rg

    # Place children within each group on inner ring(s)
    for g in groups:
        cx, cy = node_pos[g]
        pr = node_rad[g]
        children = [c for c in children_map.get(g, []) if c != g]
        if not children:
            continue
        cr_raw = [raw_radii[c] for c in children]
        if len(children) == 1:
            node_pos[children[0]] = (cx, cy)
            node_rad[children[0]] = min(pr * 0.8, cr_raw[0] * group_scale)
            continue
        ring_r = pr * 0.6
        # scale children radii to fit ring spacing
        max_cr = max(cr_raw)
        child_scale = (pr * 0.35) / (max_cr if max_cr > 0 else 1.0)
        step = 2 * math.pi / len(children)
        for i, c in enumerate(children):
            ang = i * step
            node_pos[c] = (cx + ring_r * math.cos(ang), cy + ring_r * math.sin(ang))
            node_rad[c] = max(0.02, cr_raw[i] * child_scale)

    # Any remaining nodes (deeper) place near their parent if not already placed
    for nid in df["ID"]:
        if nid in ("ROOT",) or nid in node_pos:
            continue
        parent = df.loc[df["ID"] == nid, "parentId"].values
        if len(parent) == 0:
            continue
        parent = parent[0]
        px_, py_ = node_pos.get(parent, (0.0, 0.0))
        pr_ = node_rad.get(parent, 0.2)
        node_pos[nid] = (px_ + 0.2 * pr_, py_)
        node_rad[nid] = raw_radii.get(nid, 0.05) * 0.2

    # Color scale and text colors
    color_scale = ["#0571b0", "#92c5de", "#f7f7f7", "#f4a582", "#ca0020"]
    text_colors = [
        "#2F4F4F" if term == "Reactome pathways" else ("#A9A9A9" if (pd.notna(nes_val) and float(nes_val) == 0) else "white")
        for term, nes_val in zip(df["Term"], df["nes"])
    ]
    text_color_lookup = dict(zip(df["ID"], text_colors))

    nes_values = df.set_index("ID")["nes"].to_dict()
    finite_nes = [float(v) for v in nes_values.values() if pd.notna(v)]
    vmin = min(finite_nes) if finite_nes else -1.0
    vmax = max(finite_nes) if finite_nes else 1.0
    vmin = min(vmin, -1e-6)
    vmax = max(vmax, 1e-6)

    def sample_color(val):
        if pd.isna(val):
            val = 0.0
        ratio = (float(val) - vmin) / (vmax - vmin) if vmax > vmin else 0.5
        return px.colors.sample_colorscale(color_scale, ratio)[0]

    fig = go.Figure()

    # colorbar host
    fig.add_scatter(
        x=[None],
        y=[None],
        mode="markers",
        marker=dict(
            color=[0],
            colorscale=color_scale,
            cmin=vmin,
            cmax=vmax,
            showscale=True,
            colorbar=dict(
                title=dict(text="NES", font=dict(size=16)),
                tickfont=dict(size=14),
                ticks="outside",
                thickness=20,
                len=0.6,
            ),
            size=0,
        ),
        hoverinfo="skip",
        showlegend=False,
    )

    shapes = []
    annotations = []
    for nid, (x, y) in node_pos.items():
        if nid == "ROOT":
            continue
        r = node_rad.get(nid, 0.05)
        fill = sample_color(nes_values.get(nid, 0.0))
        label = term_map.get(nid, nid)
        label_color = text_color_lookup.get(nid, "white")
        shapes.append(
            dict(
                type="circle",
                xref="x",
                yref="y",
                x0=x - r,
                y0=y - r,
                x1=x + r,
                y1=y + r,
                fillcolor=fill,
                line=dict(color="white", width=0.5),
            )
        )
        annotations.append(
            dict(
                x=x,
                y=y,
                text=f"<b>{label}</b>",
                showarrow=False,
                font=dict(size=12, color=label_color),
                xanchor="center",
                yanchor="middle",
            )
        )

    fig.update_layout(
        width=width,
        height=height,
        margin=dict(t=50, l=50, r=50, b=50),
        xaxis=dict(visible=False, scaleanchor="y", scaleratio=1, range=[-1.1, 1.1]),
        yaxis=dict(visible=False, range=[-1.1, 1.1]),
        shapes=shapes,
        annotations=annotations,
        font=dict(size=14),
        plot_bgcolor="white",
        paper_bgcolor="white",
    )

    return fig



In [7]:
geneset = pd.read_csv("/Users/polina/genetics_gsea/data/geneset_annotated/geneset_reactome_2025_disease_v2.csv")

In [149]:
# Smaller circles version: redefine plot_circle_packing with reduced radii
import math
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


def plot_circle_packing(df, width=1100, height=800, scale=0.6):
    df = df.copy()

    df["parentId"] = df["parentId"].fillna("ROOT")
    valid_ids = set(df["ID"])
    df.loc[~df["parentId"].isin(valid_ids) & (df["parentId"] != "ROOT"), "parentId"] = "ROOT"

    def wrap_text_and_orient(row):
        text = row["Term"]
        parent_id = row["parentId"]
        if text == "Reactome pathways":
            return text, "horizontal"
        orientation = "radial" if parent_id == "ROOT" else "tangential"
        if parent_id == "ROOT":
            if len(text) <= 10:
                wrapped_text = text
            elif len(text) <= 10:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = " ".join(words[:third]) + "<br>" + " ".join(words[third:2*third]) + "<br>" + " ".join(words[2*third:])
                elif len(words) == 2:
                    wrapped_text = words[0] + "<br>" + words[1]
                else:
                    wrapped_text = text
        else:
            if len(text) <= 20:
                wrapped_text = text
            elif len(text) <= 30:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = " ".join(words[:third]) + "<br>" + " ".join(words[third:3*third]) + "<br>" + " ".join(words[3*third:])
                elif len(words) == 9:
                    fourth = len(words) // 9
                    wrapped_text = " ".join(words[:fourth]) + "<br>" + " ".join(words[fourth:9*fourth]) + "<br>" + " ".join(words[9*fourth:])
                else:
                    wrapped_text = text
        return wrapped_text, orientation

    wrapped_data = df.apply(wrap_text_and_orient, axis=1, result_type="expand")
    df["wrapped_term"] = wrapped_data[0]
    df["text_orientation"] = wrapped_data[1]

    root = pd.DataFrame([{ "ID": "ROOT", "parentId": "", "wrapped_term": "Reactome pathways", "Term": "Reactome pathways", "nes": 0, "text_orientation": "horizontal" }])
    df = pd.concat([df, root], ignore_index=True)

    children_map, value_map, term_map = {}, {}, {}
    for _, row in df.iterrows():
        node_id = row["ID"]
        parent_id = row["parentId"]
        children_map.setdefault(parent_id, []).append(node_id)
        children_map.setdefault(node_id, [])
        term_map[node_id] = row["wrapped_term"]
        value_map[node_id] = float(abs(row.get("nes", 0)) if pd.notna(row.get("nes", np.nan)) else 0.0) + 0.01

    visited = set()
    def aggregate(node_id):
        if node_id in visited:
            return value_map[node_id]
        visited.add(node_id)
        if children_map.get(node_id):
            child_vals = [aggregate(c) for c in children_map[node_id]]
            value_map[node_id] = max(sum(child_vals), 0.01)
        return value_map[node_id]
    aggregate("ROOT")

    raw_radii = {nid: math.sqrt(max(value_map[nid], 1e-6)) for nid in value_map}

    groups = [nid for nid in children_map.get("ROOT", []) if nid != "ROOT"]
    if not groups:
        groups = [nid for nid in df["ID"] if nid != "ROOT"]

    group_radii = [raw_radii[g] for g in groups]
    total_r = sum(group_radii) + 1e-6
    angles = np.cumsum([0] + [2 * math.pi * (r / total_r) for r in group_radii[:-1]])

    layout_radius = 0.65
    group_scale = (layout_radius * 0.9) / (max(group_radii) if group_radii else 1.0)

    node_pos, node_rad = {}, {}
    for g, r, ang in zip(groups, group_radii, angles):
        Rg = r * group_scale
        xg = layout_radius * math.cos(ang)
        yg = layout_radius * math.sin(ang)
        node_pos[g] = (xg, yg)
        node_rad[g] = Rg

    for g in groups:
        cx, cy = node_pos[g]
        pr = node_rad[g]
        children = [c for c in children_map.get(g, []) if c != g]
        if not children:
            continue
        cr_raw = [raw_radii[c] for c in children]
        if len(children) == 1:
            node_pos[children[0]] = (cx, cy)
            node_rad[children[0]] = min(pr * 0.8, cr_raw[0] * group_scale)
            continue
        ring_r = pr * 0.6
        max_cr = max(cr_raw)
        child_scale = (pr * 0.35) / (max_cr if max_cr > 0 else 1.0)
        step = 2 * math.pi / len(children)
        for i, c in enumerate(children):
            ang = i * step
            node_pos[c] = (cx + ring_r * math.cos(ang), cy + ring_r * math.sin(ang))
            node_rad[c] = max(0.02, cr_raw[i] * child_scale)

    for nid in df["ID"]:
        if nid in ("ROOT",) or nid in node_pos:
            continue
        parent = df.loc[df["ID"] == nid, "parentId"].values
        if len(parent) == 0:
            continue
        parent = parent[0]
        px_, py_ = node_pos.get(parent, (0.0, 0.0))
        pr_ = node_rad.get(parent, 0.2)
        node_pos[nid] = (px_ + 0.2 * pr_, py_)
        node_rad[nid] = raw_radii.get(nid, 0.05) * 0.2

    # SHRINK all radii globally to make circles smaller
    shrink = float(scale)
    for nid in list(node_rad.keys()):
        node_rad[nid] *= shrink

    color_scale = ["#0571b0", "#92c5de", "#f7f7f7", "#f4a582", "#ca0020"]
    text_colors = ["#2F4F4F" if term == "Reactome pathways" else ("#A9A9A9" if (pd.notna(nes_val) and float(nes_val) == 0) else "white") for term, nes_val in zip(df["Term"], df["nes"]) ]
    text_color_lookup = dict(zip(df["ID"], text_colors))

    nes_values = df.set_index("ID")["nes"].to_dict()
    finite_nes = [float(v) for v in nes_values.values() if pd.notna(v)]
    vmin = min(finite_nes) if finite_nes else -1.0
    vmax = max(finite_nes) if finite_nes else 1.0
    vmin = min(vmin, -1e-6)
    vmax = max(vmax, 1e-6)

    def sample_color(val):
        if pd.isna(val):
            val = 0.0
        ratio = (float(val) - vmin) / (vmax - vmin) if vmax > vmin else 0.5
        return px.colors.sample_colorscale(color_scale, ratio)[0]

    fig = go.Figure()
    fig.add_scatter(x=[None], y=[None], mode="markers", marker=dict(color=[0], colorscale=color_scale, cmin=vmin, cmax=vmax, showscale=True, colorbar=dict(title=dict(text="NES", font=dict(size=16)), tickfont=dict(size=14), ticks="outside", thickness=20, len=0.6), size=0,), hoverinfo="skip", showlegend=False)

    shapes, annotations = [], []
    for nid, (x, y) in node_pos.items():
        if nid == "ROOT":
            continue
        r = node_rad.get(nid, 0.05)
        fill = sample_color(nes_values.get(nid, 0.0))
        label = term_map.get(nid, nid)
        label_color = text_color_lookup.get(nid, "white")
        shapes.append(dict(type="circle", xref="x", yref="y", x0=x - r, y0=y - r, x1=x + r, y1=y + r, fillcolor=fill, line=dict(color="white", width=0.5)))
        annotations.append(dict(x=x, y=y, text=f"<b>{label}</b>", showarrow=False, font=dict(size=12, color=label_color), xanchor="center", yanchor="middle"))

    fig.update_layout(width=width, height=height, margin=dict(t=50, l=50, r=50, b=50), xaxis=dict(visible=False, scaleanchor="y", scaleratio=1, range=[-1.1, 1.1]), yaxis=dict(visible=False, range=[-1.1, 1.1]), shapes=shapes, annotations=annotations, font=dict(size=14), plot_bgcolor="white", paper_bgcolor="white")

    return fig



In [8]:
pathways = geneset[["Term", "ID"]].drop_duplicates().reset_index(drop=True).copy()

In [9]:
pathways = pathways.assign(
    Term=pathways["Term"].str.split(";"),
    ID=pathways["ID"].str.split(";")
)

pathways_exploded = pathways.explode(["Term", "ID"]).drop_duplicates().reset_index(drop=True)

In [10]:
pathways_exploded

,Term,ID
0,Signal Transduction,R-HSA-162582
1,Generic Transcription Pathway,R-HSA-212436
2,RNA Polymerase II Transcription,R-HSA-73857
3,Gene expression (Transcription),R-HSA-74160
4,SMAD2/SMAD3:SMAD4 heterotrimer regulates trans...,R-HSA-2173796
...,...,...
403,Serotonin receptors,R-HSA-390666
404,Glycogen storage diseases,R-HSA-3229121
405,SIRT1 negatively regulates rRNA expression,R-HSA-427359
406,Alpha-defensins,R-HSA-1462054


In [11]:
hieararchy = pd.read_csv("/Users/polina/genetics_gsea/data/gmt/Reactome_2025/Pathways_hierarchy_relationship.txt", sep="\t", header=None)
hieararchy.columns = ["parentId", "childId"]

In [12]:
pathway_hieararchy = pd.merge(pathways_exploded, hieararchy, left_on="ID", right_on="childId", how="left")[["ID", "parentId", "Term"]]

In [13]:
pathway_hieararchy

,ID,parentId,Term
0,R-HSA-162582,NaN,Signal Transduction
1,R-HSA-212436,R-HSA-73857,Generic Transcription Pathway
2,R-HSA-73857,R-HSA-74160,RNA Polymerase II Transcription
3,R-HSA-74160,NaN,Gene expression (Transcription)
4,R-HSA-2173796,R-HSA-2173793,SMAD2/SMAD3:SMAD4 heterotrimer regulates trans...
...,...,...,...
408,R-HSA-390666,R-HSA-375280,Serotonin receptors
409,R-HSA-3229121,R-HSA-5663084,Glycogen storage diseases
410,R-HSA-427359,R-HSA-5250941,SIRT1 negatively regulates rRNA expression
411,R-HSA-1462054,R-HSA-1461973,Alpha-defensins


In [14]:
gsea_disease = pd.read_csv("/Users/polina/genetics_gsea/data/gsea/from_database/disease_zscore_reactome_2025.tsv", sep="\t")

In [15]:
pathway_hieararchy_gsea = pd.merge(pathway_hieararchy, gsea_disease, left_on="ID", right_on="ID", how="left")[["ID", "parentId", "Term_x", "nes", "fdr"]].rename(columns={"Term_x": "Term"})

In [16]:
import numpy as np
import plotly.express as px

In [ ]:
import pandas as pd
import plotly.express as px

def plot_sunburst(df, width=1100, height=800, show_labels=True, labels='all'):
    df = df.copy()
    
    # fill NA with ROOT
    df["parentId"] = df["parentId"].fillna("ROOT")
    
    # redirect missing parents to ROOT
    valid_ids = set(df["ID"])
    df.loc[~df["parentId"].isin(valid_ids) & (df["parentId"] != "ROOT"), "parentId"] = "ROOT"
    
    # Smart text wrapping and orientation determination (kept for wrapping only)
    def wrap_text_and_orient(row):
        text = row["Term"]
        parent_id = row["parentId"]
        
        if text == "Reactome pathways":
            return text, "horizontal"  # center label
        
        if parent_id == "ROOT":
            orientation = "radial"
        else:
            orientation = "tangential"
        
        # Apply text wrapping
        if parent_id == "ROOT":
            if len(text) <= 10:
                wrapped_text = text
            elif len(text) <= 10:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = " ".join(words[:third]) + "<br>" + " ".join(words[third:2*third]) + "<br>" + " ".join(words[2*third:])
                elif len(words) == 2:
                    wrapped_text = words[0] + "<br>" + words[1]
                else:
                    wrapped_text = text
        else:
            if len(text) <= 20:
                wrapped_text = text
            elif len(text) <= 30:
                words = text.split()
                if len(words) >= 2:
                    mid = len(words) // 2
                    wrapped_text = " ".join(words[:mid]) + "<br>" + " ".join(words[mid:])
                else:
                    wrapped_text = text
            else:
                words = text.split()
                if len(words) >= 3:
                    third = len(words) // 3
                    wrapped_text = " ".join(words[:third]) + "<br>" + " ".join(words[third:3*third]) + "<br>" + " ".join(words[3*third:])
                elif len(words) == 9:
                    fourth = len(words) // 9
                    wrapped_text = " ".join(words[:fourth]) + "<br>" + " ".join(words[fourth:9*fourth]) + "<br>" + " ".join(words[9*fourth:])
                else:
                    wrapped_text = text
        
        return wrapped_text, orientation
    
    wrapped_data = df.apply(wrap_text_and_orient, axis=1, result_type='expand')
    df["wrapped_term"] = wrapped_data[0]
    df["text_orientation"] = wrapped_data[1]
    
    # add root node
    root = pd.DataFrame([{
        "ID": "ROOT", 
        "parentId": "", 
        "wrapped_term": "Reactome pathways", 
        "Term": "Reactome pathways", 
        "nes": 0,
        "text_orientation": "horizontal"
    }])
    df = pd.concat([df, root], ignore_index=True)

    # Colorblind-friendly diverging scale
    color_scale = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020']
    
        # Per-node text colors: dark grey for center, grey for nes==0, white otherwise
    text_colors = [
        '#2F4F4F' if term == "Reactome pathways" else ('#A9A9A9' if (pd.notna(nes_val) and float(nes_val) == 0) else 'white')
        for term, nes_val in zip(df["Term"], df["nes"])
    ]

    # Hide center label; we'll render a custom annotation instead
    df.loc[df["ID"].eq("ROOT"), "wrapped_term"] = " " 

    # Backward-compat: if show_labels is False, force labels mode to 'none'
    labels = ('none' if not show_labels else labels)
    if labels not in {'all', 'none', 'root_only'}:
        labels = 'all'

    # Compute per-node label text based on desired mode
    if labels == 'none':
        df['label_to_show'] = ''
    elif labels == 'root_only':
        df['label_to_show'] = df.apply(lambda r: (r['wrapped_term'] if (r['parentId'] == 'ROOT' and r['ID'] != 'ROOT') else ''), axis=1)
    else:  # 'all'
        df['label_to_show'] = df.apply(lambda r: ('' if r['ID'] == 'ROOT' else r['wrapped_term']), axis=1)
    
    fig = px.sunburst(
        df,
        names="wrapped_term",
        ids="ID",
        parents="parentId",
        values=None,
        color="nes",
        color_continuous_scale=color_scale,
        color_continuous_midpoint=0,
        branchvalues='total',
        width=width,
        height=height
    )
    
    fig.update_layout(
        margin=dict(t=50, l=50, r=50, b=50),
        coloraxis_colorbar=dict(
            title=dict(text="NES", font=dict(size=16)),
            tickvals=[-5, -2.5, 0, 2.5, 5],
            # tickvals=[df["nes"].min(), 0, df["nes"].max()],
            tickfont=dict(size=14),
            ticks="outside",
            thickness=20,
            len=0.6
        ),
        font=dict(size=14)
    )


    # Let Plotly choose radial/tangential per sector; center stays horizontal.
    fig.update_traces(
        insidetextorientation='auto',
        textfont=dict(size=14, family="Arial", color=text_colors),
        hovertemplate='<b>%{customdata[0]}</b><br>NES: %{color:.2f}<extra></extra>',
        customdata=df[["Term"]],
        text=df['label_to_show'],
        textinfo='text',
        texttemplate='<b>%{text}</b>'
    )
    
    return fig


In [155]:
import pandas as pd

# Filter pathways by FDR
pathway_hieararchy_gsea_filt = pathway_hieararchy_gsea[pathway_hieararchy_gsea["fdr"] < 0.005].copy()

# Fill missing parentId with ROOT
pathway_hieararchy_gsea_filt["parentId"] = pathway_hieararchy_gsea_filt["parentId"].fillna("ROOT")

# Recursively remove rows where parentId is not in ID column (except for ROOT)
def filter_orphaned_nodes(df):
    valid_ids = set(df["ID"])
    
    # Keep only rows where parentId is in valid IDs or is ROOT
    filtered_df = df[df["parentId"].isin(valid_ids.union({"ROOT"}))].copy()
    
    # If no changes were made, we're done
    if len(filtered_df) == len(df):
        return filtered_df
    
    # Otherwise, continue filtering recursively
    return filter_orphaned_nodes(filtered_df)

# Apply the recursive filtering
pathway_hieararchy_gsea_filt_vis = filter_orphaned_nodes(pathway_hieararchy_gsea_filt)

# Add new row for 'Other' using pd.concat
other_row = pd.DataFrame([{"ID": "Other", "Term": "Other", "nes": -5, "parentId": "ROOT"}])
pathway_hieararchy_gsea_filt_vis = pd.concat([pathway_hieararchy_gsea_filt_vis, other_row], ignore_index=True)

# Optional: save to CSV
# pathway_hieararchy_gsea_filt_vis.to_csv("/Users/polina/genetics_gsea/data/sunburst/pathway_hieararchy_gsea_filt.csv", index=False)

# If you want to exclude the 'Other' category from the final result
pathway_hieararchy_gsea_filt_vis_no_other = pathway_hieararchy_gsea_filt_vis[
    (pathway_hieararchy_gsea_filt_vis["parentId"] != "Other")
]

In [156]:
pathway_hieararchy_gsea_filt_vis_no_other

,ID,parentId,Term,nes,fdr
0,R-HSA-162582,ROOT,Signal Transduction,5.879025,0.000007
1,R-HSA-212436,R-HSA-73857,Generic Transcription Pathway,4.456707,0.000382
2,R-HSA-73857,R-HSA-74160,RNA Polymerase II Transcription,4.202785,0.000634
3,R-HSA-74160,ROOT,Gene expression (Transcription),4.022728,0.000928
4,R-HSA-1280215,R-HSA-168256,Cytokine Signaling in Immune system,4.523809,0.000357
5,R-HSA-168256,ROOT,Immune System,4.273209,0.000557
6,R-HSA-1266738,ROOT,Developmental Biology,4.117902,0.000737
7,R-HSA-1643685,ROOT,Disease,3.556823,0.003337
8,R-HSA-449147,R-HSA-1280215,Signaling by Interleukins,3.410020,0.004928
9,R-HSA-8963743,ROOT,Digestion and absorption,-3.887837,0.001404


In [157]:
fig = plot_sunburst(pathway_hieararchy_gsea_filt_vis_no_other)
fig.show()

In [21]:
fig.write_image("/Users/polina/genetics_gsea/data/sunburst/sunburst_plot_v1.png", width=1100, height=800, scale=2)

# Plot parents that were not enriched in grey

In [129]:
# Filter pathways by FDR
pathway_hieararchy_gsea_001 = pathway_hieararchy_gsea[pathway_hieararchy_gsea["fdr"] < 0.001].copy()

In [151]:
hieararchy = pd.read_csv("/Users/polina/genetics_gsea/data/gmt/Reactome_2025/Pathways_hierarchy_relationship.txt", sep="\t", header=None)

In [153]:
hieararchy.rename(columns={0: "parentId", 1: "childId"}, inplace=True)

In [132]:
import numpy as np

# Recreate full hierarchical tree for pathway_hieararchy_gsea_001 by adding ancestor rows
# Build child -> parent mapping from the Reactome hierarchy table
# Expecting `hieararchy` with columns ["parentId", "childId"] already loaded
child_to_parent = dict(zip(hieararchy["childId"], hieararchy["parentId"]))

base_df = pathway_hieararchy_gsea_001.copy()

# Start with all rows in base_df
rows_to_add = []
existing_ids = set(base_df["ID"].astype(str))

# For each row, climb up the parent chain and add missing ancestor rows
for _, row in base_df.iterrows():
    current_parent = row["parentId"]
    visited = set()
    while isinstance(current_parent, str) and current_parent and (current_parent not in visited):
        visited.add(current_parent)
        # Determine this parent's parent from the mapping (if any)
        next_parent = child_to_parent.get(current_parent, None)
        # Add the row for the current parent if it's not already present
        if current_parent not in existing_ids:
            rows_to_add.append({
                "ID": current_parent,
                "parentId": next_parent if isinstance(next_parent, str) else np.nan,
                "Term": np.nan,
                "nes": 0,
                "fdr": np.nan
            })
            existing_ids.add(current_parent)
        # Move up the chain
        current_parent = next_parent

# Append new rows, if any, and reset index
if rows_to_add:
    ancestors_df = pd.DataFrame(rows_to_add, columns=["ID", "parentId", "Term", "nes", "fdr"]).drop_duplicates()
    pathway_hieararchy_gsea_001_expanded = pd.concat([base_df, ancestors_df], ignore_index=True)
else:
    pathway_hieararchy_gsea_001_expanded = base_df.copy()

pathway_hieararchy_gsea_001_expanded

,ID,parentId,Term,nes,fdr
0,R-HSA-162582,NaN,Signal Transduction,5.879025,0.000007
1,R-HSA-212436,R-HSA-73857,Generic Transcription Pathway,4.456707,0.000382
2,R-HSA-73857,R-HSA-74160,RNA Polymerase II Transcription,4.202785,0.000634
3,R-HSA-74160,NaN,Gene expression (Transcription),4.022728,0.000928
4,R-HSA-1280215,R-HSA-168256,Cytokine Signaling in Immune system,4.523809,0.000357
...,...,...,...,...,...
226,R-HSA-5250941,R-HSA-212165,NaN,0.000000,NaN
227,R-HSA-1461973,R-HSA-6803157,NaN,0.000000,NaN
228,R-HSA-6803157,R-HSA-168249,NaN,0.000000,NaN
229,R-HSA-425397,R-HSA-425407,NaN,0.000000,NaN


In [133]:
# Fill Term only for rows where Term is NaN, using complete_list_of_pathways
complete_path = "/Users/polina/genetics_gsea/data/gmt/Reactome_2025/complete_list_of_pathways.txt"
complete_df = pd.read_csv(complete_path, sep="\t", header=None, names=["ID", "Term", "extra"], dtype=str)[["ID", "Term"]]

# Choose source table (prefer expanded; else base filtered)
try:
    target_df = pathway_hieararchy_gsea_001_expanded.copy()
except NameError:
    target_df = pathway_hieararchy_gsea_001.copy()

# Ensure strings
target_df["ID"] = target_df["ID"].astype(str)
complete_df["ID"] = complete_df["ID"].astype(str)

# Build mapping and fill only NaN Terms
term_map = dict(zip(complete_df["ID"], complete_df["Term"]))
mask_na_term = target_df["Term"].isna()
target_df.loc[mask_na_term, "Term"] = target_df.loc[mask_na_term, "ID"].map(term_map)

# Output updated table without adding new IDs
pathway_hieararchy_gsea_001_full = target_df
pathway_hieararchy_gsea_001_full


,ID,parentId,Term,nes,fdr
0,R-HSA-162582,NaN,Signal Transduction,5.879025,0.000007
1,R-HSA-212436,R-HSA-73857,Generic Transcription Pathway,4.456707,0.000382
2,R-HSA-73857,R-HSA-74160,RNA Polymerase II Transcription,4.202785,0.000634
3,R-HSA-74160,NaN,Gene expression (Transcription),4.022728,0.000928
4,R-HSA-1280215,R-HSA-168256,Cytokine Signaling in Immune system,4.523809,0.000357
...,...,...,...,...,...
226,R-HSA-5250941,R-HSA-212165,Negative epigenetic regulation of rRNA expression,0.000000,NaN
227,R-HSA-1461973,R-HSA-6803157,Defensins,0.000000,NaN
228,R-HSA-6803157,R-HSA-168249,Antimicrobial peptides,0.000000,NaN
229,R-HSA-425397,R-HSA-425407,"Transport of vitamins, nucleosides, and relate...",0.000000,NaN


In [134]:
fig2 = plot_sunburst(pathway_hieararchy_gsea_001_full, width=1100, height=1100)
fig2.show()

In [ ]:
# fig2.write_image("/Users/polina/genetics_gsea/data/sunburst/sunburst_plot_v4.png", width=1100, height=1100, scale=2)

In [115]:
# # Save fig2 to HTML
# output_html = "/Users/polina/genetics_gsea/data/sunburst/sunburst_plot_v1.html"
# fig2.write_html(output_html, include_plotlyjs='cdn')
# output_html

'/Users/polina/genetics_gsea/data/sunburst/sunburst_plot_v2.html'

In [137]:
fig3 = plot_flame_horizontal(pathway_hieararchy_gsea_001_full, width=1100, height=800)
fig3.show()

In [26]:
from pyecharts.charts import Sunburst
from pyecharts import options as opts
import panel as pn
pn.extension('echarts')

data = [
    {
        "name": "Flora",
        "itemStyle": {"color": "#da0d68"},
        "children": [
            {"name": "Black Tea", "value": 1, "itemStyle": {"color": "#975e6d"}},
            {
                "name": "Floral",
                "itemStyle": {"color": "#e0719c"},
                "children": [
                    {
                        "name": "Chamomile",
                        "value": 1,
                        "itemStyle": {"color": "#f99e1c"},
                    },
                    {"name": "Rose", "value": 1, "itemStyle": {"color": "#ef5a78"}},
                    {"name": "Jasmine", "value": 1, "itemStyle": {"color": "#f7f1bd"}},
                ],
            },
        ],
    },
    {
        "name": "Fruity",
        "itemStyle": {"color": "#da1d23"},
        "children": [
            {
                "name": "Berry",
                "itemStyle": {"color": "#dd4c51"},
                "children": [
                    {
                        "name": "Blackberry",
                        "value": 1,
                        "itemStyle": {"color": "#3e0317"},
                    },
                    {
                        "name": "Raspberry",
                        "value": 1,
                        "itemStyle": {"color": "#e62969"},
                    },
                    {
                        "name": "Blueberry",
                        "value": 1,
                        "itemStyle": {"color": "#6569b0"},
                    },
                    {
                        "name": "Strawberry",
                        "value": 1,
                        "itemStyle": {"color": "#ef2d36"},
                    },
                ],
            },
            {
                "name": "Dried Fruit",
                "itemStyle": {"color": "#c94a44"},
                "children": [
                    {"name": "Raisin", "value": 1, "itemStyle": {"color": "#b53b54"}},
                    {"name": "Prune", "value": 1, "itemStyle": {"color": "#a5446f"}},
                ],
            },
            {
                "name": "Other Fruit",
                "itemStyle": {"color": "#dd4c51"},
                "children": [
                    {"name": "Coconut", "value": 1, "itemStyle": {"color": "#f2684b"}},
                    {"name": "Cherry", "value": 1, "itemStyle": {"color": "#e73451"}},
                    {
                        "name": "Pomegranate",
                        "value": 1,
                        "itemStyle": {"color": "#e65656"},
                    },
                    {
                        "name": "Pineapple",
                        "value": 1,
                        "itemStyle": {"color": "#f89a1c"},
                    },
                    {"name": "Grape", "value": 1, "itemStyle": {"color": "#aeb92c"}},
                    {"name": "Apple", "value": 1, "itemStyle": {"color": "#4eb849"}},
                    {"name": "Peach", "value": 1, "itemStyle": {"color": "#f68a5c"}},
                    {"name": "Pear", "value": 1, "itemStyle": {"color": "#baa635"}},
                ],
            },
            {
                "name": "Citrus Fruit",
                "itemStyle": {"color": "#f7a128"},
                "children": [
                    {
                        "name": "Grapefruit",
                        "value": 1,
                        "itemStyle": {"color": "#f26355"},
                    },
                    {"name": "Orange", "value": 1, "itemStyle": {"color": "#e2631e"}},
                    {"name": "Lemon", "value": 1, "itemStyle": {"color": "#fde404"}},
                    {"name": "Lime", "value": 1, "itemStyle": {"color": "#7eb138"}},
                ],
            },
        ],
    },
    {
        "name": "Sour/\nFermented",
        "itemStyle": {"color": "#ebb40f"},
        "children": [
            {
                "name": "Sour",
                "itemStyle": {"color": "#e1c315"},
                "children": [
                    {
                        "name": "Sour Aromatics",
                        "value": 1,
                        "itemStyle": {"color": "#9ea718"},
                    },
                    {
                        "name": "Acetic Acid",
                        "value": 1,
                        "itemStyle": {"color": "#94a76f"},
                    },
                    {
                        "name": "Butyric Acid",
                        "value": 1,
                        "itemStyle": {"color": "#d0b24f"},
                    },
                    {
                        "name": "Isovaleric Acid",
                        "value": 1,
                        "itemStyle": {"color": "#8eb646"},
                    },
                    {
                        "name": "Citric Acid",
                        "value": 1,
                        "itemStyle": {"color": "#faef07"},
                    },
                    {
                        "name": "Malic Acid",
                        "value": 1,
                        "itemStyle": {"color": "#c1ba07"},
                    },
                ],
            },
            {
                "name": "Alcohol/\nFremented",
                "itemStyle": {"color": "#b09733"},
                "children": [
                    {"name": "Winey", "value": 1, "itemStyle": {"color": "#8f1c53"}},
                    {"name": "Whiskey", "value": 1, "itemStyle": {"color": "#b34039"}},
                    {
                        "name": "Fremented",
                        "value": 1,
                        "itemStyle": {"color": "#ba9232"},
                    },
                    {"name": "Overripe", "value": 1, "itemStyle": {"color": "#8b6439"}},
                ],
            },
        ],
    },
    {
        "name": "Green/\nVegetative",
        "itemStyle": {"color": "#187a2f"},
        "children": [
            {"name": "Olive Oil", "value": 1, "itemStyle": {"color": "#a2b029"}},
            {"name": "Raw", "value": 1, "itemStyle": {"color": "#718933"}},
            {
                "name": "Green/\nVegetative",
                "itemStyle": {"color": "#3aa255"},
                "children": [
                    {
                        "name": "Under-ripe",
                        "value": 1,
                        "itemStyle": {"color": "#a2bb2b"},
                    },
                    {"name": "Peapod", "value": 1, "itemStyle": {"color": "#62aa3c"}},
                    {"name": "Fresh", "value": 1, "itemStyle": {"color": "#03a653"}},
                    {
                        "name": "Dark Green",
                        "value": 1,
                        "itemStyle": {"color": "#038549"},
                    },
                    {
                        "name": "Vegetative",
                        "value": 1,
                        "itemStyle": {"color": "#28b44b"},
                    },
                    {"name": "Hay-like", "value": 1, "itemStyle": {"color": "#a3a830"}},
                    {
                        "name": "Herb-like",
                        "value": 1,
                        "itemStyle": {"color": "#7ac141"},
                    },
                ],
            },
            {"name": "Beany", "value": 1, "itemStyle": {"color": "#5e9a80"}},
        ],
    },
    {
        "name": "Other",
        "itemStyle": {"color": "#0aa3b5"},
        "children": [
            {
                "name": "Papery/Musty",
                "itemStyle": {"color": "#9db2b7"},
                "children": [
                    {"name": "Stale", "value": 1, "itemStyle": {"color": "#8b8c90"}},
                    {
                        "name": "Cardboard",
                        "value": 1,
                        "itemStyle": {"color": "#beb276"},
                    },
                    {"name": "Papery", "value": 1, "itemStyle": {"color": "#fefef4"}},
                    {"name": "Woody", "value": 1, "itemStyle": {"color": "#744e03"}},
                    {
                        "name": "Moldy/Damp",
                        "value": 1,
                        "itemStyle": {"color": "#a3a36f"},
                    },
                    {
                        "name": "Musty/Dusty",
                        "value": 1,
                        "itemStyle": {"color": "#c9b583"},
                    },
                    {
                        "name": "Musty/Earthy",
                        "value": 1,
                        "itemStyle": {"color": "#978847"},
                    },
                    {"name": "Animalic", "value": 1, "itemStyle": {"color": "#9d977f"}},
                    {
                        "name": "Meaty Brothy",
                        "value": 1,
                        "itemStyle": {"color": "#cc7b6a"},
                    },
                    {"name": "Phenolic", "value": 1, "itemStyle": {"color": "#db646a"}},
                ],
            },
            {
                "name": "Chemical",
                "itemStyle": {"color": "#76c0cb"},
                "children": [
                    {"name": "Bitter", "value": 1, "itemStyle": {"color": "#80a89d"}},
                    {"name": "Salty", "value": 1, "itemStyle": {"color": "#def2fd"}},
                    {
                        "name": "Medicinal",
                        "value": 1,
                        "itemStyle": {"color": "#7a9bae"},
                    },
                    {
                        "name": "Petroleum",
                        "value": 1,
                        "itemStyle": {"color": "#039fb8"},
                    },
                    {"name": "Skunky", "value": 1, "itemStyle": {"color": "#5e777b"}},
                    {"name": "Rubber", "value": 1, "itemStyle": {"color": "#120c0c"}},
                ],
            },
        ],
    },
    {
        "name": "Roasted",
        "itemStyle": {"color": "#c94930"},
        "children": [
            {"name": "Pipe Tobacco", "value": 1, "itemStyle": {"color": "#caa465"}},
            {"name": "Tobacco", "value": 1, "itemStyle": {"color": "#dfbd7e"}},
            {
                "name": "Burnt",
                "itemStyle": {"color": "#be8663"},
                "children": [
                    {"name": "Acrid", "value": 1, "itemStyle": {"color": "#b9a449"}},
                    {"name": "Ashy", "value": 1, "itemStyle": {"color": "#899893"}},
                    {"name": "Smoky", "value": 1, "itemStyle": {"color": "#a1743b"}},
                    {
                        "name": "Brown, Roast",
                        "value": 1,
                        "itemStyle": {"color": "#894810"},
                    },
                ],
            },
            {
                "name": "Cereal",
                "itemStyle": {"color": "#ddaf61"},
                "children": [
                    {"name": "Grain", "value": 1, "itemStyle": {"color": "#b7906f"}},
                    {"name": "Malt", "value": 1, "itemStyle": {"color": "#eb9d5f"}},
                ],
            },
        ],
    },
    {
        "name": "Spices",
        "itemStyle": {"color": "#ad213e"},
        "children": [
            {"name": "Pungent", "value": 1, "itemStyle": {"color": "#794752"}},
            {"name": "Pepper", "value": 1, "itemStyle": {"color": "#cc3d41"}},
            {
                "name": "Brown Spice",
                "itemStyle": {"color": "#b14d57"},
                "children": [
                    {"name": "Anise", "value": 1, "itemStyle": {"color": "#c78936"}},
                    {"name": "Nutmeg", "value": 1, "itemStyle": {"color": "#8c292c"}},
                    {"name": "Cinnamon", "value": 1, "itemStyle": {"color": "#e5762e"}},
                    {"name": "Clove", "value": 1, "itemStyle": {"color": "#a16c5a"}},
                ],
            },
        ],
    },
    {
        "name": "Nutty/\nCocoa",
        "itemStyle": {"color": "#a87b64"},
        "children": [
            {
                "name": "Nutty",
                "itemStyle": {"color": "#c78869"},
                "children": [
                    {"name": "Peanuts", "value": 1, "itemStyle": {"color": "#d4ad12"}},
                    {"name": "Hazelnut", "value": 1, "itemStyle": {"color": "#9d5433"}},
                    {"name": "Almond", "value": 1, "itemStyle": {"color": "#c89f83"}},
                ],
            },
            {
                "name": "Cocoa",
                "itemStyle": {"color": "#bb764c"},
                "children": [
                    {
                        "name": "Chocolate",
                        "value": 1,
                        "itemStyle": {"color": "#692a19"},
                    },
                    {
                        "name": "Dark Chocolate",
                        "value": 1,
                        "itemStyle": {"color": "#470604"},
                    },
                ],
            },
        ],
    },
    {
        "name": "Sweet",
        "itemStyle": {"color": "#e65832"},
        "children": [
            {
                "name": "Brown Sugar",
                "itemStyle": {"color": "#d45a59"},
                "children": [
                    {"name": "Molasses", "value": 1, "itemStyle": {"color": "#310d0f"}},
                    {
                        "name": "Maple Syrup",
                        "value": 1,
                        "itemStyle": {"color": "#ae341f"},
                    },
                    {
                        "name": "Caramelized",
                        "value": 1,
                        "itemStyle": {"color": "#d78823"},
                    },
                    {"name": "Honey", "value": 1, "itemStyle": {"color": "#da5c1f"}},
                ],
            },
            {"name": "Vanilla", "value": 1, "itemStyle": {"color": "#f89a80"}},
            {"name": "Vanillin", "value": 1, "itemStyle": {"color": "#f37674"}},
            {"name": "Overall Sweet", "value": 1, "itemStyle": {"color": "#e75b68"}},
            {"name": "Sweet Aromatics", "value": 1, "itemStyle": {"color": "#d0545f"}},
        ],
    },
]


c = Sunburst(init_opts=opts.InitOpts(width="1000px", height="600px")).add(
    "",
    data_pair=data,
    highlight_policy="ancestor",
    radius=[0, "95%"],
    sort_="null",
    levels=[
        {},
        {
            "r0": "15%",
            "r": "35%",
            "itemStyle": {"borderWidth": 2},
            "label": {"rotate": "tangential"},
        },
        {"r0": "35%", "r": "70%", "label": {"align": "right"}},
        {
            "r0": "70%",
            "r": "72%",
            "label": {"position": "outside", "padding": 3, "silent": False},
            "itemStyle": {"borderWidth": 3},
        },
    ],
)

pn.Row(c).show()

/var/folders/xs/llnz2_gn4fsfwjd4c0prr1w00000gp/T/ipykernel_6097/3047841338.py:4: UserWarning:

Using Panel interactively in VSCode notebooks requires the jupyter_bokeh package to be installed. You can install it with:

   pip install jupyter_bokeh

or:
    conda install jupyter_bokeh

and try again.



Launching server at http://localhost:62536
